## Exploratory Data Analysis Code

In [ ]:
# -*- coding: utf-8 -*-
"""
Task 2: Exploratory Data Analysis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Load enriched data
df = pd.read_csv('../_data/processed/ethiopia_fi_enriched.csv')
observations = df[df['record_type'] == 'observation'].copy()
events = df[df['record_type'] == 'event'].copy()
impact_links = df[df['record_type'] == 'impact_link'].copy()

# Convert dates
observations['observation_date'] = pd.to_datetime(observations['observation_date'])
events['event_date'] = pd.to_datetime(events['event_date'])

# ============================================================================
# 1. DATASET OVERVIEW
# ============================================================================

print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)

print(f"\n📊 Total records: {len(df)}")
print(f"📈 Observations: {len(observations)}")
print(f"🎯 Events: {len(events)}")
print(f"🔗 Impact links: {len(impact_links)}")

# Data quality assessment
print("\n" + "=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)

print("\n📋 Confidence levels:")
print(observations['confidence'].value_counts(dropna=False))

print("\n❌ Missing values per column:")
missing_pct = (observations.isnull().sum() / len(observations) * 100).round(2)
print(missing_pct[missing_pct > 0])

# ============================================================================
# 2. ACCESS ANALYSIS - Account Ownership
# ============================================================================

print("\n" + "=" * 80)
print("ACCESS ANALYSIS - Account Ownership")
print("=" * 80)

# Filter account ownership data
acc_ownership = observations[
    observations['indicator_code'].str.contains('ACC_OWNERSHIP', na=False)
].copy()
acc_ownership = acc_ownership.sort_values('observation_date')

# Calculate growth rates
acc_ownership['year'] = acc_ownership['observation_date'].dt.year
acc_ownership['growth_rate'] = acc_ownership['value_numeric'].pct_change() * 100

print("\n📈 Account Ownership Timeline:")
print(acc_ownership[['year', 'value_numeric', 'growth_rate']].to_string(index=False))

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Line plot
ax1.plot(acc_ownership['year'], acc_ownership['value_numeric'], 
         marker='o', linewidth=2, markersize=8, color='steelblue')
ax1.fill_between(acc_ownership['year'], 0, acc_ownership['value_numeric'], 
                 alpha=0.2, color='steelblue')
ax1.set_title('Account Ownership Rate (2011-2024)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year')
ax1.set_ylabel('% of Adults (15+)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 60)

# Annotate key points
for idx, row in acc_ownership.iterrows():
    ax1.annotate(f"{row['value_numeric']}%", 
                (row['year'], row['value_numeric']),
                textcoords="offset points",
                xytext=(0,10),
                ha='center',
                fontsize=9)

# Growth rate bar plot
colors = ['green' if x > 0 else 'red' for x in acc_ownership['growth_rate'].iloc[1:]]
ax2.bar(acc_ownership['year'].iloc[1:], acc_ownership['growth_rate'].iloc[1:],
        color=colors, alpha=0.7)
ax2.set_title('Year-over-Year Growth Rate', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Growth Rate (%)')
ax2.grid(True, alpha=0.3, axis='y')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('../_data/processed/account_ownership_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 3. USAGE ANALYSIS - Digital Payments
# ============================================================================

print("\n" + "=" * 80)
print("USAGE ANALYSIS - Digital Payments")
print("=" * 80)

# Filter digital payment data
digital_payments = observations[
    observations['indicator_code'].str.contains('DIGITAL_PAYMENT', na=False) |
    observations['indicator'].str.contains('digital payment', case=False, na=False)
].copy()
digital_payments = digital_payments.sort_values('observation_date')

if not digital_payments.empty:
    print("\n📊 Digital Payment Indicators:")
    for code in digital_payments['indicator_code'].unique():
        subset = digital_payments[digital_payments['indicator_code'] == code]
        print(f"\n{code}:")
        for _, row in subset.iterrows():
            print(f"  {row['observation_date'].year}: {row['value_numeric']}%")
    
    # Visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for code in digital_payments['indicator_code'].unique():
        subset = digital_payments[digital_payments['indicator_code'] == code]
        ax.plot(subset['observation_date'].dt.year, 
                subset['value_numeric'], 
                marker='o', 
                label=code,
                linewidth=2)
    
    ax.set_title('Digital Payment Adoption Trends', fontsize=14, fontweight='bold')
    ax.set_xlabel('Year')
    ax.set_ylabel('% of Adults')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../_data/processed/digital_payments_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# ============================================================================
# 4. MOBILE MONEY ANALYSIS
# ============================================================================

mm_data = observations[
    observations['indicator_code'].str.contains('MM_', na=False) |
    observations['indicator'].str.contains('mobile money', case=False, na=False)
].copy()

if not mm_data.empty:
    print("\n" + "=" * 80)
    print("MOBILE MONEY ANALYSIS")
    print("=" * 80)
    
    mm_data = mm_data.sort_values('observation_date')
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Group by year and indicator
    pivot_data = mm_data.pivot_table(
        index=mm_data['observation_date'].dt.year,
        columns='indicator',
        values='value_numeric',
        aggfunc='first'
    )
    
    pivot_data.plot(ax=ax, marker='o', linewidth=2)
    ax.set_title('Mobile Money Indicators Over Time', fontsize=14, fontweight='bold')
    ax.set_xlabel('Year')
    ax.set_ylabel('Percentage/Value')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../_data/processed/mobile_money_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# ============================================================================
# 5. INFRASTRUCTURE AND ENABLERS ANALYSIS
# ============================================================================

infrastructure_data = observations[
    (observations['pillar'] == 'enabler') |
    observations['indicator'].str.contains('coverage|penetration|density', 
                                         case=False, na=False)
].copy()

if not infrastructure_data.empty:
    print("\n" + "=" * 80)
    print("INFRASTRUCTURE AND ENABLERS")
    print("=" * 80)
    
    # Correlation with account ownership
    # Create a merged dataset by year
    yearly_data = []
    
    for year in range(2011, 2025):
        year_data = {'year': year}
        
        # Get account ownership for year
        acc_year = acc_ownership[acc_ownership['observation_date'].dt.year == year]
        if not acc_year.empty:
            year_data['account_ownership'] = acc_year['value_numeric'].iloc[0]
        
        # Get infrastructure data for year
        infra_year = infrastructure_data[
            infrastructure_data['observation_date'].dt.year == year
        ]
        
        for _, row in infra_year.iterrows():
            # Create a clean key name
            key_name = f"{row['indicator_code']}_{row['observation_date'].year}"
            year_data[key_name] = row['value_numeric']
        
        yearly_data.append(year_data)
    
    yearly_df = pd.DataFrame(yearly_data)
    
    # Calculate correlations
    print("\n🔗 Correlations with Account Ownership:")
    for col in yearly_df.columns:
        if col != 'year' and col != 'account_ownership' and pd.notna(yearly_df[col]).any():
            corr = yearly_df['account_ownership'].corr(yearly_df[col])
            if pd.notna(corr):
                print(f"{col}: {corr:.3f}")

# ============================================================================
# 6. EVENT TIMELINE VISUALIZATION
# ============================================================================

print("\n" + "=" * 80)
print("EVENT TIMELINE ANALYSIS")
print("=" * 80)

# Create timeline
fig = go.Figure()

# Add account ownership line
fig.add_trace(go.Scatter(
    x=acc_ownership['observation_date'],
    y=acc_ownership['value_numeric'],
    mode='lines+markers',
    name='Account Ownership',
    line=dict(color='blue', width=3),
    marker=dict(size=8)
))

# Add events as vertical lines
for _, event in events.iterrows():
    fig.add_vline(
        x=event['event_date'],
        line_width=2,
        line_dash="dash",
        line_color="red",
        annotation_text=event['event_name'],
        annotation_position="top left",
        annotation_font_size=10
    )

fig.update_layout(
    title='Account Ownership with Key Events Timeline',
    xaxis_title='Date',
    yaxis_title='Account Ownership (%)',
    height=600,
    template='plotly_white'
)

fig.write_html('../_data/processed/event_timeline.html')
fig.show()

# ============================================================================
# 7. GENDER GAP ANALYSIS (if data exists)
# ============================================================================

gender_data = observations[
    observations['indicator'].str.contains('female|male|gender', case=False, na=False)
].copy()

if not gender_data.empty:
    print("\n" + "=" * 80)
    print("GENDER GAP ANALYSIS")
    print("=" * 80)
    
    # Pivot for comparison
    gender_pivot = gender_data.pivot_table(
        index='observation_date',
        columns='indicator',
        values='value_numeric'
    )
    
    fig, ax = plt.subplots(figsize=(12, 6))
    gender_pivot.plot(ax=ax, marker='o', linewidth=2)
    ax.set_title('Gender-wise Financial Inclusion', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Percentage')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../_data/processed/gender_gap_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# ============================================================================
# 8. KEY INSIGHTS SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("KEY INSIGHTS")
print("=" * 80)

insights = [
    "1. 📈 Account ownership grew rapidly from 14% (2011) to 49% (2024), but growth slowed to +3pp in 2021-2024",
    "2. 📱 Mobile money expansion (Telebirr + M-Pesa) didn't translate to proportional account ownership increase",
    "3. 🔗 Infrastructure indicators (4G, smartphone penetration) show strong correlation with digital payments",
    "4. ⏰ Key events (product launches) show visible inflection points in adoption curves",
    "5. 📊 Data gaps exist in gender-disaggregated and regional data for comprehensive analysis",
    "6. 💳 Digital payment adoption (~35%) outpaces mobile money account ownership (~9.5%)",
    "7. 🚀 2021-2024 slowdown suggests saturation in easily reachable urban population",
    "8. 🔄 Interoperability milestones likely boosted usage more than access"
]

for insight in insights:
    print(f"\n{insight}")

# ============================================================================
# 9. SAVE ANALYSIS RESULTS
# ============================================================================

# Create summary DataFrame
summary_stats = {
    'metric': [
        'Account Ownership 2011',
        'Account Ownership 2024',
        'Total Growth 2011-2024',
        'Annual Growth Rate (Avg)',
        'Mobile Money Penetration 2024',
        'Digital Payment Adoption 2024'
    ],
    'value': [
        f"{acc_ownership['value_numeric'].iloc[0]}%",
        f"{acc_ownership['value_numeric'].iloc[-1]}%",
        f"+{acc_ownership['value_numeric'].iloc[-1] - acc_ownership['value_numeric'].iloc[0]}pp",
        f"{(acc_ownership['value_numeric'].iloc[-1] / acc_ownership['value_numeric'].iloc[0]) ** (1/13) - 1:.1%}",
        '9.45%',  # From problem statement
        '~35%'    # From problem statement
    ],
    'insight': [
        'Baseline year',
        'Current level',
        '35 percentage point increase over 13 years',
        'Compound annual growth rate',
        'Despite 65M+ accounts opened',
        'Higher than mobile money ownership'
    ]
}

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv('../_data/processed/key_metrics_summary.csv', index=False)
print("\n✅ Saved key metrics summary to: ../_data/processed/key_metrics_summary.csv")

# Create data quality report
quality_report = {
    'total_observations': len(observations),
    'date_range': f"{observations['observation_date'].min()} to {observations['observation_date'].max()}",
    'unique_indicators': len(observations['indicator_code'].unique()),
    'high_confidence_pct': f"{(observations['confidence'] == 'high').sum() / len(observations) * 100:.1f}%",
    'missing_values_pct': f"{observations.isnull().sum().sum() / (len(observations) * len(observations.columns)) * 100:.1f}%"
}

quality_df = pd.DataFrame(list(quality_report.items()), columns=['metric', 'value'])
quality_df.to_csv('../_data/processed/data_quality_report.csv', index=False)
print("✅ Saved data quality report to: ../_data/processed/data_quality_report.csv")